# Evaluate warbleR autodetect
The goal of this notebook is to evaluate performance of the `autodetect` method against the labelled data set provided by Stefan

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from birdutils import read_labels

We start by developing algorithm that can assess the accuracy. We take the labelled data set and compare it against one of the output files produced by `autodetect`.

In [2]:
bird_calls_labels_path = '/mnt/data/Birdman/sthelena_labels.xls'
detected_calls_path = '/mnt/data/Birdman/samples/features/autodetect/autodetect_petrels_bp1-8_wl256_th2_smooth1200_maxdur4.csv'

In [3]:
labels_dict = read_labels(bird_calls_labels_path)
labels = pd.concat(labels_dict, axis=0, ignore_index=True)
detected = pd.read_csv(detected_calls_path, usecols=['sound.files', 'selec', 'duration', 'start', 'end'])

In [4]:
labels.head()

,Date,File Name,Type of Call,Time Start,Time End,Species,Notes
0,2014-06-05,STHELENA-02_20140605_200000_1,Flight Call,2.905,4.197,Storm Petrel,NaN
1,2014-06-05,STHELENA-02_20140605_200000_1,Flight Call,10.008,11.622,Storm Petrel,NaN
2,2014-06-05,STHELENA-02_20140605_200000_1,Flight Call,14.527,16.464,Storm Petrel,NaN
3,2014-06-05,STHELENA-02_20140605_200000_1,Flight Call,17.110,18.724,Storm Petrel,NaN
4,2014-06-05,STHELENA-02_20140605_200000_1,Flight Call,20.338,21.307,Storm Petrel,NaN


In [5]:
detected.head()

,sound.files,selec,duration,start,end
0,STHELENA-02_20140605_200000_10.wav,1,0.208063,20.171815,20.379877
1,STHELENA-02_20140605_200000_10.wav,2,0.816938,20.538940,21.355877
2,STHELENA-02_20140605_200000_10.wav,3,0.223563,21.368252,21.591815
3,STHELENA-02_20140605_200000_10.wav,4,0.603438,22.350002,22.953440
4,STHELENA-02_20140605_200000_10.wav,5,0.308938,39.672442,39.981379


In [6]:
detected = detected.rename(columns={'sound.files': 'File Name', 'start': 'Time Start', 'end': 'Time End'})
detected['File Name'] = detected['File Name'].str.replace('.wav', '')

In [7]:
detected.head()

,File Name,selec,duration,Time Start,Time End
0,STHELENA-02_20140605_200000_10,1,0.208063,20.171815,20.379877
1,STHELENA-02_20140605_200000_10,2,0.816938,20.538940,21.355877
2,STHELENA-02_20140605_200000_10,3,0.223563,21.368252,21.591815
3,STHELENA-02_20140605_200000_10,4,0.603438,22.350002,22.953440
4,STHELENA-02_20140605_200000_10,5,0.308938,39.672442,39.981379


In [8]:
def get_acc(features, labels):
    features['overlap'] = False
    labels['overlap'] = False
    filenames = labels['File Name'].unique()

    for filename in filenames:
        df_label = labels[labels['File Name'] == filename]
        df_feat = features[features['File Name'] == filename]
        for index1, row_label in df_label.iterrows():
            for index2, row_feat in df_feat.iterrows():
                feat_start = row_feat['Time Start']
                feat_end = row_feat['Time End']
                label_start = row_label['Time Start']
                label_end = row_label['Time End']
                overlap = (label_start <= feat_end) and (label_end >= feat_start)
                if overlap:
                    labels.at[index1, 'overlap'] = True
                    features.at[index2, 'overlap'] = True
    return sum(labels['overlap']) / len(labels)

In [9]:
print(f'Accuracy for {bird_calls_labels_path}: {get_acc(detected, labels) * 100:.2f}%') 

Accuracy for /mnt/data/Birdman/sthelena_labels.xls: 70.36%


## Get accuracy for all tested parameters
In WarbleR `autodetect` we can tune various parameters like:
* threshold
* amplitude envelope smoothing
* ... and many more to be found in [docs](https://www.rdocumentation.org/packages/warbleR/versions/1.1.14/topics/autodetec)

We put the params in the file name and then check which one perform best.

In [10]:
detected_all_paths = glob.glob('/mnt/data/Birdman/samples/features/autodetect/autodetect_*.csv')

In [11]:
for path in detected_all_paths:
    name = os.path.basename(path)
    detected = pd.read_csv(path, usecols=['sound.files', 'selec', 'duration', 'start', 'end'])
    detected = detected.rename(columns={'sound.files': 'File Name', 'start': 'Time Start', 'end': 'Time End'})
    detected['File Name'] = detected['File Name'].str.replace('.wav', '')
    acc = get_acc(detected, labels)
    print(f'Accuracy of detected calls in {name}: {acc*100:.2f}%')

Accuracy of detected calls in autodetect_petrels_bp1-8_wl512_th1_smooth1200_maxdur12.csv: 43.30%
Accuracy of detected calls in autodetect_petrels_bp1-8_wl256_th2_smooth1200_maxdur4.csv: 70.36%
Accuracy of detected calls in autodetect_petrels_bp1-8_wl512_th10_smooth1200_maxdur4.csv: 21.13%
Accuracy of detected calls in autodetect_petrels_bp1-8_wl256_th1_smooth1200_maxdur4.csv: 49.23%
Accuracy of detected calls in autodetect_petrels_bp1-8_wl256_th2_smooth600.csv: 31.96%
Accuracy of detected calls in autodetect_petrels_bp1-12_wl256_th2_smooth1200_maxdur4_hil.csv: 70.36%
Accuracy of detected calls in autodetect_petrels_bp1-8_wl256_th2_smooth1200_mdur04.csv: 31.44%
Accuracy of detected calls in autodetect_petrels_bp1-8_wl512_th1_smooth500_maxdur4.csv: 45.62%
Accuracy of detected calls in autodetect_petrels_bp0-8_wl256_th2_smooth1200_maxdur4_hil.csv: 1.03%
Accuracy of detected calls in autodetect_petrels_bp1-8_wl256_th2_smooth2000_maxdur4_hil.csv: 62.11%
Accuracy of detected calls in autodet

## Results
Clearly the results aren't impressive. 30% of calls goes unnoticed.

In [12]:
# det_df.to_csv('detected.csv', index=False)